In [1]:
import openai
import llm2geneset
import pandas as pd
import math
import random
import gc

aclient = openai.AsyncClient()

In [2]:
al002 = pd.read_csv("data/Stats.TECAN.AL002_vs_isotype.txt", sep="\t")
al002 = al002[al002["PercentDetectedSamples"] > 0.9]

In [3]:
al002_sig = al002[(al002["padj"] < 0.1) ]
genes = al002_sig["gene"].to_list()

In [4]:
random.seed(138337)
random.shuffle(genes)

In [5]:
print("\n".join(genes))

LILRB2
MX1
GSN
MFGE8
CCL3
GPD1
TRPV2
DHCR24
TNFAIP3
MATK
CXCL8
CHIT1
TPRKB
RASSF4
SMC6
FOS
RHOBTB2
RNF19B
FBP1
SLC11A1
DCSTAMP
HSD3B7
OAS2
TGM2
MCEMP1
FAM20C
ACP5
CD9
AP5B1


In [6]:
context_str = ""
llm2geneset_no_context_res = await llm2geneset.gs_proposal_bench(aclient, [genes], model="gpt-4o-2024-05-13", context=context_str,
                                                                 n_pathways=100)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:25<00:00, 25.52s/it]


In [7]:
df = llm2geneset_no_context_res[0]["ora_results"]
df.to_csv("outputs/no_context_AL002.tsv",sep="\t",index=None)
df[df["p_adj"] < 0.01]

,set_descr,generatio,bgratio,richFactor,foldEnrich,p_val,p_adj,intersection,set_genes,ngenes,nset,ninter,in_toks,out_toks


In [8]:
context_str = "in vitro microglia treated with a TREM2 agonist antibody"
llm2geneset_context_res = await llm2geneset.gs_proposal_bench(aclient, [genes], model="gpt-4o-2024-05-13", 
                                                              context=context_str,
                                                              n_pathways=100)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:28<00:00, 28.74s/it]


In [9]:
df = llm2geneset_context_res[0]["ora_results"]
df.to_csv("outputs/with_context_AL002.tsv",sep="\t",index=None)
df[df["p_adj"] < 0.01]

,set_descr,generatio,bgratio,richFactor,foldEnrich,p_val,p_adj,intersection,set_genes,ngenes,nset,ninter,in_toks,out_toks
64,Myeloid Cell Activation,0.103448,0.001512,0.1,68.434483,0.000011,0.001092,"CXCL8,TNFAIP3,CCL3","CSF2,ITGB2,TLR2,JAK2,CXCR1,CXCL8,MPL,TLR4,STAT...",29,30,3,158,394


In [10]:
gmt = llm2geneset.read_gmt("libs_human/gmt/KEGG_2021_Human.txt")
set_descr = gmt["descr"]
gene_sets = gmt["genes"]
df = llm2geneset.simple_ora(genes, set_descr, gene_sets)
df.to_csv("outputs/KEGG_AL002.tsv",sep="\t",index=None)
df[df["p_adj"] < 0.05]

,set_descr,generatio,bgratio,richFactor,foldEnrich,p_val,p_adj,intersection,set_genes,ngenes,nset,ninter
251,Rheumatoid arthritis,0.137931,0.004686,0.043011,29.434186,0.000010,0.003157,"CXCL8,ACP5,CCL3,FOS","CSF2,HLA-DMB,HLA-DOA,TCIRG1,FOS,TGFB1,ANGPT1,T...",29,93,4
161,Measles,0.137931,0.007004,0.028777,19.693376,0.000048,0.007686,"MX1,TNFAIP3,OAS2,FOS","SLAMF1,OAS1,IKBKB,IL12B,FOS,CASP3,CD3E,BAK1,NF...",29,139,4
173,NOD-like receptor signaling pathway,0.137931,0.009120,0.022099,15.123643,0.000134,0.014265,"CXCL8,TNFAIP3,OAS2,TRPV2","MAPK12,OAS1,IKBKB,CARD9,CASP12,TRPM2,ERBIN,XIA...",29,181,4
136,IL-17 signaling pathway,0.103448,0.004736,0.031915,21.840792,0.000345,0.021238,"CXCL8,TNFAIP3,FOS","USP25,CSF2,CXCL10,MAPK12,IL17B,IKBKB,FOS,CEBPB...",29,94,3
64,Coronavirus disease,0.137931,0.011690,0.017241,11.799049,0.000345,0.021238,"CXCL8,MX1,OAS2,FOS","CFB,CSF2,CXCL10,RPS11,MAPK12,RPL3L,OAS1,IKBKB,...",29,232,4
50,Chagas disease,0.103448,0.005140,0.029412,20.127789,0.000439,0.021238,"CXCL8,CCL3,FOS","MAPK12,IKBKB,IL12B,FOS,TGFB1,PPP2R2A,CD3E,NFKB...",29,102,3
290,Toll-like receptor signaling pathway,0.103448,0.005240,0.028846,19.740716,0.000465,0.021238,"CXCL8,CCL3,FOS","CXCL10,MAPK12,IKBKB,IL12B,FOS,CCL4L1,TLR5,NFKB...",29,104,3
191,Osteoclast differentiation,0.103448,0.006399,0.023622,16.165626,0.000832,0.033260,"ACP5,LILRB2,FOS","MAPK12,NOX1,IKBKB,FOS,TGFB1,OSCAR,NFKBIA,MAPK8...",29,127,3


In [11]:
#
ppmi_olink_p196 = pd.read_csv("data/alt_ppmi_csf_PDvsHC.lm.20241004.tsv", sep="\t")
ppmi_olink_p196 = ppmi_olink_p196[(ppmi_olink_p196["wt_pvalue"]<1e-2)]# & ((ppmi_olink_p196["l2fc"])>0)]
#
# ppmi_olink_p196 = pd.read_csv("data/covariates_ppmi_csf_PDvsHC.lm20241008.tsv", sep="\t")
# print(ppmi_olink_p196.columns)
# ppmi_olink_p196 = ppmi_olink_p196[(ppmi_olink_p196["wt_pvalue"]<0.1) & (ppmi_olink_p196["l2fc"]<0)]
#
genes = ppmi_olink_p196["GENE"].to_list()
#
random.seed(20241009)
random.shuffle(genes)
#
print(",".join(genes))


PTX3,VWF,JUN,PRDX5,MGLL,SNAP23,CCL22,APBB1IP,LBR,KRT19,SKAP2,ARG1,INPP1,S100A11,PSG1,SH2B3,CXCL11,PLA2G2A,GSAP,XG,PDGFB,IL1RN,EGLN1,TFRC,OMG,PM20D1,CCN4,FBP1,ST6GAL1,IL5,ALPP,SMAD5,SPINK5,NADK,CCL26,LEFTY2,ITGB2,PI3,SERPINA12,LTA4H,FCAR,CIAPIN1,CA13,GH1,MPI,CST7,PSPN,NCF2,SERPINE1,DPY30,PAEP,C19orf12,CRACR2A,ANXA11,LRRC25,ADGRE5,PARP1,MASP1,CCL3,MZB1,F7,MMP3,CLC,NUB1,SLAMF7,VMO1,PPME1,IDUA,CDHR1,DDC,KLB,GOLM2,SPINK4,TRIM21,CLIP2,TNF,ABHD14B,MSLN,ENPP5,KLK4,APEX1,ERBIN,RHOC,ANXA3,CCL23,CES3,PFKFB2,RET,CCL28,BTC,PPY,NOS3,MNDA,WFIKKN1,ITGA6,LHB,SRP14,CDHR2,LEP,CLEC4A,LY75,LAT,KIR3DL1


In [12]:
#
gmt = llm2geneset.read_gmt("libs_human/gmt/KEGG_2021_Human.txt")
set_descr = gmt["descr"]
gene_sets = gmt["genes"]
df = llm2geneset.simple_ora(genes, set_descr, gene_sets)
df.to_csv("outputs/KEGG_ppmi-olink-p196.tsv",sep="\t",index=None)
df[df["p_adj"] < 0.01]


,set_descr,generatio,bgratio,richFactor,foldEnrich,p_val,p_adj,intersection,set_genes,ngenes,nset,ninter
68,Cytokine-cytokine receptor interaction,0.106796,0.014864,0.037288,7.184663,4.057176e-07,0.000130,"IL1RN,CCL22,CCL28,CCL26,CXCL11,GH1,LEP,TNF,CCL...","IL20,CSF2,CXCL10,IL23R,CCR3,TNFRSF18,TNFSF4,IL...",103,295,11
308,Viral protein interaction with cytokine and cy...,0.067961,0.005039,0.070000,13.487573,9.208993e-07,0.000147,"CCL22,CCL28,CCL26,CXCL11,TNF,CCL23,CCL3","IL20,CXCL10,CCR3,IL37,IL6R,IL34,CCL4L1,CCL1,CC...",103,100,7
63,Complement and coagulation cascades,0.048544,0.004283,0.058824,11.334095,8.257319e-05,0.008808,"ITGB2,F7,SERPINE1,VWF,MASP1","CFHR5,CFB,CD59,FGB,PLG,F9,SERPINB2,C5,C4A,C7,S...",103,85,5


In [13]:
#
gc.collect()
context_str = ""
llm2geneset_no_context_res = await llm2geneset.gs_proposal_bench(aclient, 
                                                                 [genes], 
                                                                 model="gpt-4o-2024-05-13", 
                                                                 context=context_str,
                                                                 n_pathways=100)

df = llm2geneset_no_context_res[0]["ora_results"]
df.to_csv("outputs/no_context_ppmi-olink-p196.tsv",sep="\t",index=None)
df[df["p_adj"] < 0.01]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:24<00:00, 24.49s/it]


,set_descr,generatio,bgratio,richFactor,foldEnrich,p_val,p_adj,intersection,set_genes,ngenes,nset,ninter,in_toks,out_toks
6,Chemokine-mediated signaling pathway,0.058252,0.004132,0.073171,14.098508,0.000004,0.000429,"CCL22,CCL28,CCL26,CXCL11,CCL23,CCL3","CXCL10,CCR3,PDPK1,MAPK8,CCL1,CCL19,GNAI1,CCL21...",103,82,6,160,740
53,Annexin-mediated pathways,0.029126,0.000705,0.214286,41.288488,0.000048,0.002335,"S100A11,ANXA11,ANXA3","ANXA1,ANXA5,ANXA11,ANXA3,ANXA8,S100A10,S100A11...",103,14,3,158,190
58,Coagulation cascade,0.029126,0.001260,0.120000,23.121553,0.000292,0.009434,"SERPINE1,F7,VWF","F2,F12,VWF,SERPINC1,PLG,F9,SERPINF2,F11,F10,PR...",103,25,3,156,320
82,Ret signaling pathway,0.029126,0.001411,0.107143,20.644244,0.000411,0.009964,"RET,JUN,PSPN","RET,MAPK3,GFRA3,DOK5,MAPK1,STAT3,PTPN11,NRTN,C...",103,28,3,154,364


In [14]:
#
gc.collect()
context_str = "human cerebrospinal fluid"
llm2geneset_context_res = await llm2geneset.gs_proposal_bench(aclient, [genes], model="gpt-4o-2024-05-13", 
                                                              context=context_str,
                                                              n_pathways=100)

df = llm2geneset_context_res[0]["ora_results"]
df.to_csv("outputs/with_lesscontext_ppmi-olink-p196.tsv",sep="\t",index=None)
df[df["p_adj"] < 0.01]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:45<00:00, 45.90s/it]


,set_descr,generatio,bgratio,richFactor,foldEnrich,p_val,p_adj,intersection,set_genes,ngenes,nset,ninter,in_toks,out_toks
60,Chemokine-mediated signaling pathway,0.058252,0.005895,0.051282,9.881006,0.000034,0.003205,"CCL22,CCL28,CCL26,CXCL11,CCL23,CCL3","CXCR7,CXCL10,CCR3,STAT6,GNG5,CCL1,CCL19,GNAI1,...",103,117,6,160,1062
1,Wound healing,0.029126,0.001008,0.150000,28.901942,0.000147,0.005994,"JUN,TNF,PDGFB","FGF2,ANGPT1,PDGFB,SERPINB2,ITGB1,SPARC,VEGFA,F...",103,20,3,154,264
87,Extracellular region,0.038835,0.002771,0.072727,14.013063,0.000194,0.005994,"MMP3,SERPINE1,VWF,PDGFB","PECAM1,ICAM4,FGF2,THBS3,FBN1,VWF,CDH1,TGFB1,TI...",103,55,4,156,715
23,Leukocyte chemotaxis,0.038835,0.002973,0.067797,13.063025,0.000255,0.005994,"ITGB2,TNF,CCL3,CXCL11","PECAM1,MAPK3,CXCL3,CXCL10,ITGB2,TLR2,Selectin ...",103,59,4,160,546


In [15]:
#
gc.collect()
context_str = "Parkinson's disease"
llm2geneset_context_res = await llm2geneset.gs_proposal_bench(aclient, [genes], model="gpt-4o-2024-05-13", 
                                                              context=context_str,
                                                              n_pathways=100)

df = llm2geneset_context_res[0]["ora_results"]
df[df["p_adj"] < 0.01]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:31<00:00, 31.59s/it]


,set_descr,generatio,bgratio,richFactor,foldEnrich,p_val,p_adj,intersection,set_genes,ngenes,nset,ninter,in_toks,out_toks
92,Chemokine-mediated signaling pathway,0.058252,0.003426,0.088235,17.001142,0.000001,0.000141,"CCL22,CCL28,CCL26,CXCL11,CCL23,CCL3","CXCR7,CXCL3,CCL11,CCR3,CXCR1,CXCR3,CXCL10,CXCL...",103,68,6,160,612
75,Vascular permeability,0.038835,0.002469,0.081633,15.728948,0.000124,0.005932,"JUN,NOS3,TNF,PDGFB","PECAM1,CDH5,MTOR,KDR,PIK3CA,FOS,PLG,ANGPT1,PDG...",103,49,4,154,632


In [16]:
#
gc.collect()
# context_str = "cerebrospinal fluid of Parkinson's disease patients based on Olink proteomics"
context_str = "cerebrospinal fluid in Parkinson's disease"
llm2geneset_context_res = await llm2geneset.gs_proposal_bench(aclient, [genes], model="gpt-4o-2024-05-13", 
                                                              context=context_str,
                                                              n_pathways=100)

df = llm2geneset_context_res[0]["ora_results"]
df.to_csv("outputs/with_context_ppmi-olink-p196.tsv",sep="\t",index=None)
df[df["p_adj"] < 0.01]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:32<00:00, 32.35s/it]


,set_descr,generatio,bgratio,richFactor,foldEnrich,p_val,p_adj,intersection,set_genes,ngenes,nset,ninter,in_toks,out_toks
15,Chemokine-Mediated Signaling Pathway,0.058252,0.003376,0.089552,17.254891,0.000001,0.000130,"CCL22,CCL28,CCL26,CXCL11,CCL23,CCL3","CXCR7,CXCL3,CCL11,CCR3,CXCR1,CXCR3,CXCL10,CXCL...",103,67,6,168,604
38,Wound Healing,0.038835,0.001612,0.125000,24.084951,0.000022,0.001089,"SERPINE1,NOS3,TNF,PDGFB","FGF2,CDH1,CXCL12,ANGPT1,PDGFB,CCL2,ITGB1,NOS3,...",103,32,4,154,420
12,Endothelial Cell Activation,0.029126,0.000756,0.200000,38.535922,0.000060,0.001938,"TNF,NOS3,PDGFB","TEK,RELA,CXCL8,NFKB1,ANGPT2,TNF,PTGS2,VCAM1,IL...",103,15,3,158,196
18,Acute Inflammatory Response,0.029126,0.001260,0.120000,23.121553,0.000292,0.007075,"IL1RN,SERPINE1,TNF","MAPK3,CSF2,S100A8,CXCL8,TLR4,MAPK1,STAT3,CCR5,...",103,25,3,158,327
33,Coagulation,0.029126,0.001411,0.107143,20.644244,0.000411,0.007971,"SERPINE1,F7,VWF","F2,APOH,F12,VWF,FGB,SERPINC1,PLG,KNG1,F9,SERPI...",103,28,3,154,355


### Pseudobulk of scRNAseq from Substantia Nigra - a PD atlas dataset

In [56]:
#
mgl_scpdsn_df = pd.read_csv("data/Stats.scRNA_pseudobulk_microglia.PD_vs_Healthy.txt", sep="\t")
# Filtering for significants DEGs post multi-testing
mgl_scpdsn_sig = mgl_scpdsn_df[(mgl_scpdsn_df["padj"] < 0.05)]


In [57]:
#
mgl_scpdsn_genes = mgl_scpdsn_sig["Gene"].to_list()
random.seed(999999)
random.shuffle(mgl_scpdsn_genes)

print(",".join(mgl_scpdsn_genes))

NEFL,KIF3A,GNG3,ZFP36L1,GABRG2,TAGLN3,FTL,PCBD2,NFX1,CKB,FGFR1OP2,PRKCQ,PKIB,EXOC5,DHTKD1,SULT4A1,ARHGDIG,TYW1,CLEC17A,ELAVL2,SORT1,THAP2,ZNF714,AL139220.2,LRP10,TPT1,LEPROTL1,ITGB2-AS1,SULT1A1,ALDH1A1,ZC3H14,KLHL1,ZNF839,CADM3,SESN1,BNIP3L,CLEC1A,PPA2,CDKL1,LSMEM1,MARCO,FXYD5,CD247,LINC00847,NUPR1,TTN,FGF12,MTPN,COLEC12,HP1BP3,C9orf153,EMB,HDGF,RALGAPA2,DOK6,RPL17,SLC6A3,CHPT1,LINC02397,HIST2H2BE,DAPK2,BMI1,LINC00862,LMO3,UNC13C,SYCP2,ASMTL-AS1,KLHL21,AKAP12,SERPINB9,BIN2,LMF1,MITD1,MIA2,MID1IP1,TERF2IP,GRIA3,WASF3,VAV3,LYG1,ANAPC16,TSEN54,DOC2A,CDKN2D,DPH6,NFATC2IP,ARHGEF9,PSMD9,ATP6V0E2,SLC18A2,ZNF25,MAGOH,PPP3CB,CRYGS,TOR1AIP2,RETREG1,RAB6A,PAQR6,CSF3R,LUCAT1,CIRBP,PPID,MSH3,KCNQ3,CLDN12,ZBTB44,RIMS1,ACOT7,AC083949.1,INO80,DCTN4,BIN1,HIST1H2BG,L2HGDH,DTWD2,BEX1,ENOSF1,RELA,TMEM170A,PAPOLA,HTT,ZNF706,NHSL1,CARNS1,RSBN1,IER2,LINC01877,EMP2,DCUN1D4,ARG2,FSCN1,NAIP,AC233296.1,SV2B,TANC2,KIF21A,F13A1,PEMT,CCDC26,ANKRD13A,ARSG,AATK,MICU2,RSPO2,TAZ,KLHDC4,SYT1,KCNB1,SNX22,PTPN12,HIVEP3,IT

In [58]:
#
gmt = llm2geneset.read_gmt("libs_human/gmt/KEGG_2021_Human.txt")
set_descr = gmt["descr"]
gene_sets = gmt["genes"]
df = llm2geneset.simple_ora(mgl_scpdsn_genes, set_descr, gene_sets)
df.to_csv("outputs/KEGG_scRNA_mgl_PD.tsv",sep="\t",index=None)
df[df["p_adj"] < 0.01]


,set_descr,generatio,bgratio,richFactor,foldEnrich,p_val,p_adj,intersection,set_genes,ngenes,nset,ninter


In [59]:
#
gc.collect()
#
context_str = ""
llm2geneset_no_context_res = await llm2geneset.gs_proposal_bench(aclient, 
                                                                 [mgl_scpdsn_genes], 
                                                                 model="gpt-4o-2024-05-13", 
                                                                 context=context_str,
                                                                 n_pathways=100)

df = llm2geneset_no_context_res[0]["ora_results"]
df.to_csv("outputs/no_context_scRNA_mgl_PD.tsv",sep="\t",index=None)
df[df["p_adj"] < 0.01]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:26<00:00, 26.88s/it]


,set_descr,generatio,bgratio,richFactor,foldEnrich,p_val,p_adj,intersection,set_genes,ngenes,nset,ninter,in_toks,out_toks


In [54]:
#
gc.collect()
#
context_str = "Rheumatoid arthritis"
llm2geneset_context_res = await llm2geneset.gs_proposal_bench(aclient, [mgl_scpdsn_genes], model="gpt-4o-2024-05-13", 
                                                              context=context_str,
                                                              n_pathways=100)

df = llm2geneset_context_res[0]["ora_results"]
df.to_csv("outputs/wrong_context_scRNA_mgl_PD.tsv",sep="\t",index=None)
df[df["p_adj"] < 0.01]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:25<00:00, 25.96s/it]


,set_descr,generatio,bgratio,richFactor,foldEnrich,p_val,p_adj,intersection,set_genes,ngenes,nset,ninter,in_toks,out_toks


In [55]:
#
gc.collect()
#
context_str = "differentially expressed genes from pseudobulk of microglia in Parkinson's disease"
llm2geneset_context_res = await llm2geneset.gs_proposal_bench(aclient, [mgl_scpdsn_genes], model="gpt-4o-2024-05-13", 
                                                              context=context_str,
                                                              n_pathways=100)

df = llm2geneset_context_res[0]["ora_results"]
df.to_csv("outputs/with_context_scRNA_mgl_PD.tsv",sep="\t",index=None)
df[df["p_adj"] < 0.01]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:27<00:00, 27.02s/it]


,set_descr,generatio,bgratio,richFactor,foldEnrich,p_val,p_adj,intersection,set_genes,ngenes,nset,ninter,in_toks,out_toks
41,Dopaminergic synapse,0.047619,0.00136,0.111111,35.001764,0.000085,0.008013,"SLC6A3,TH,SLC18A2","NTRK2,DRD3,DRD5,GRK6,PRKN,PPP1R1B,PINK1,DJ1,VM...",63,27,3,160,246
